Purpose:
This notebook is run after "01_import_CPA_boundaries.ipynb"

# INPUT
It is looking from CSV downloaded from Get It Done website.

# OUTPUT
Drainage related requests that have passed basic QA: gid_drainage_candidate_last30.csv 
Records without address or coordinates which need to be reviewed: gid_drainage_review_last30.csv rows: 30

This notebook has the following steps:
- Use pandas to import the Get It Done complaints from the city of San Diego
- Filter them to drainage-related data
- Do some minor address cleanup
- Check for any outlying data points outside San Diego or rows with missing coordinates
- Standardize address and coordinate cleanup status message

In [28]:
import os
import pandas as pd
import arcpy
import requests

arcpy.env.overwriteOutput = True

PROJECT_ROOT = r"C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone"

RAW_DIR = os.path.join(PROJECT_ROOT, "data_raw")
WORK_DIR = os.path.join(PROJECT_ROOT, "data_working")
GDB_PATH = os.path.join(WORK_DIR, "GetItDoneAnalysis.gdb")

# Inputs - Pull the CSV from the website
url = "https://seshat.datasd.org/get_it_done_reports/get_it_done_requests_open_datasd.csv"
GETITDONE_CSV = os.path.join(RAW_DIR, "get_it_done_requests_open_datasd.csv")

response = requests.get(url)
response.raise_for_status()  # stops if download fails

with open(GETITDONE_CSV, "wb") as f:
    f.write(response.content)

print(f"Saved {GETITDONE_CSV}")

# Optional: set workspace for easier listing
arcpy.env.workspace = GDB_PATH

for p in [GETITDONE_CSV]:
    if not os.path.exists(p) and not arcpy.Exists(p):
        raise FileNotFoundError(f"Missing required input: {p}")

print("CSV:", GETITDONE_CSV)
print("GDB:", GDB_PATH)

Saved C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_raw\get_it_done_requests_open_datasd.csv
CSV: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_raw\get_it_done_requests_open_datasd.csv
GDB: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\GetItDoneAnalysis.gdb


In [29]:
df = pd.read_csv(GETITDONE_CSV)

print("Rows:", len(df))
print("Columns:", list(df.columns))
df.head(3)


Rows: 89909
Columns: ['service_request_id', 'service_request_parent_id', 'sap_notification_number', 'date_requested', 'case_age_days', 'case_record_type', 'service_name', 'service_name_detail', 'date_closed', 'status', 'lat', 'lng', 'street_address', 'zipcode', 'council_district', 'comm_plan_code', 'comm_plan_name', 'park_name', 'case_origin', 'referred', 'iamfloc', 'floc', 'public_description']


,service_request_id,service_request_parent_id,sap_notification_number,date_requested,case_age_days,case_record_type,service_name,service_name_detail,date_closed,status,...,zipcode,council_district,comm_plan_code,comm_plan_name,park_name,case_origin,referred,iamfloc,floc,public_description
0,100763,NaN,4.030001e+10,2016-08-20T14:46:00,3439,TSW,Street Sweeping,NaN,NaN,In Process,...,NaN,9.0,56.0,Mid-City:City Heights,NaN,Web,NaN,SS-014304,SS-001240,A) The storm drain channel south of 5135 Univ...
1,100777,NaN,4.030001e+10,2016-08-20T15:48:00,3439,TSW,Sidewalk Repair Issue,SIDEWALK MINOR REHAB CONTRACT,NaN,In Process,...,NaN,9.0,59.0,Mid-City:Normal Heights,NaN,Web,NaN,SS-000917-SE1,SS-000917,Curb in rubble
2,100985,NaN,4.030001e+10,2016-08-22T10:04:00,3437,TSW,Stormwater,DRAIN HEADWALL,NaN,In Process,...,NaN,6.0,15.0,Mira Mesa,NaN,Phone,NaN,HW01082,SS-019619,HILLSIDE ERODING - POSSIBLE BROKEN DRAIN


In [30]:
# Lowercase helper columns
df["service_name_lc"] = df["service_name"].astype(str).str.lower()
df["service_detail_lc"] = df["service_name_detail"].astype(str).str.lower()

# Start broad, refine later
drainage_mask = (
    df["service_name_lc"].str.contains("storm", na=False) |
    df["service_name_lc"].str.contains("drain", na=False) |
    df["service_detail_lc"].str.contains("storm", na=False) |
    df["service_detail_lc"].str.contains("drain", na=False) |
    df["service_detail_lc"].str.contains("flood", na=False)
)

dr = df[drainage_mask].copy()
print("Drainage candidate rows:", len(dr))

# Peek at the most common categories to tighten the filter
dr["service_name"].value_counts().head(15)


Drainage candidate rows: 3367


service_name
Stormwater                     2275
Stormwater Code Enforcement     976
ROW Maintenance                 113
Parks Issue                       3
Name: count, dtype: int64

In [31]:
# Ensure numeric
dr["lat_num"] = pd.to_numeric(dr["lat"], errors="coerce")
dr["lng_num"] = pd.to_numeric(dr["lng"], errors="coerce")

dr["qa_missing_coords"] = dr["lat_num"].isna() | dr["lng_num"].isna()
dr["qa_missing_address"] = dr["street_address"].isna() | (dr["street_address"].astype(str).str.strip() == "")

# Basic plausible range check (San Diego-ish bounding box)
# (This is a quick sanity check, not a precise boundary test)
dr["qa_coords_out_of_range"] = ~(
    dr["lat_num"].between(32.5, 33.2, inclusive="both") &
    dr["lng_num"].between(-117.4, -116.8, inclusive="both")
)

print("Missing coords:", int(dr["qa_missing_coords"].sum()))
print("Missing address:", int(dr["qa_missing_address"].sum()))
print("Coords out of range:", int(dr["qa_coords_out_of_range"].sum()))


Missing coords: 27
Missing address: 0
Coords out of range: 30


In [32]:
# Create a status message showing errors found in the rows (if any)
def qa_status(row):
    if row["qa_missing_coords"]:
        return "MISSING_COORDS"
    if row["qa_coords_out_of_range"]:
        return "OUT_OF_RANGE"
    if row["qa_missing_address"]:
        return "MISSING_ADDRESS"
    return "OK"

dr["QA_STATUS"] = dr.apply(qa_status, axis=1)

# quick check
dr["QA_STATUS"].value_counts(dropna=False)


QA_STATUS
OK                3337
MISSING_COORDS      27
OUT_OF_RANGE         3
Name: count, dtype: int64

In [33]:
# Good enough to attempt CPA join: has coords and coords are plausible
dr["qa_join_candidate"] = (~dr["qa_missing_coords"]) & (~dr["qa_coords_out_of_range"])

# Everything else is "review"
dr_review = dr[~dr["qa_join_candidate"]].copy()
dr_candidate = dr[dr["qa_join_candidate"]].copy()

print("Candidate for CPA join:", len(dr_candidate))
print("Review (missing/out-of-range):", len(dr_review))

Candidate for CPA join: 3337
Review (missing/out-of-range): 30


In [34]:
# Some basic cleanup of punctuation and addresses
dr["street_address_clean"] = (
    dr["street_address"]
    .astype(str)
    .str.upper()
    # Find every period (.) in the string and remove it
    .str.replace(r"\.", "", regex=True)
    # Replace any run of multiple spaces, tabs, or line breaks with a single space
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)


In [35]:
# Write two CSV files - one that has records with coordinates, and one to be reviewed that doesn't

# Imports and environment setups
import os
import arcpy

arcpy.env.overwriteOutput = True

# --- Set project root folder 
PROJECT_ROOT = r"C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone"
REVIEW_CSV = os.path.join(WORK_DIR, "gid_drainage_review_last30.csv")

def export_for_xy(df_in, out_csv):
    out = df_in.copy()
    out["lat"] = pd.to_numeric(out["lat"], errors="coerce")
    out["lon"] = pd.to_numeric(out["lng"], errors="coerce")  # adjust if lng field name differs
    # Keep desired columns; make sure QA_STATUS is included
    keep = [c for c in [
        "service_request_id", "service_name", "service_name_detail",
        "public_description", "date_requested", "status",
        "street_address", "street_address_clean",
        "QA_STATUS"
    ] if c in out.columns] + ["lat", "lon"]
    out[keep].to_csv(out_csv, index=False)
    print("Wrote:", out_csv, "rows:", len(out))

export_for_xy(dr_candidate, CANDIDATE_CSV)
export_for_xy(dr_review, REVIEW_CSV)

Wrote: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\gid_drainage_candidate_last30.csv rows: 3337
Wrote: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\gid_drainage_review_last30.csv rows: 30
